In [62]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import difflib 

In [63]:
from fuzzywuzzy import process


In [64]:
#read in the data
data_tests = pd.read_csv('data-private-school-tests.csv')
data_tests 


,School,Test,# tested,% proficient
0,A FANTIS PAROCHIAL,Grade 3 ELA,11,100.00%
1,A FANTIS PAROCHIAL,Grade 3 Math,11,100.00%
2,A FANTIS PAROCHIAL,Grade 4 ELA,11,100.00%
3,A FANTIS PAROCHIAL,Grade 4 Math,11,90.91%
4,A FANTIS PAROCHIAL,Grade 5 ELA,14,92.86%
...,...,...,...,...
4969,YESODEI BINA INC,Grade 5 ELA,3,0.00%
4970,YESODEI BINA INC,Grade 5 Math,1,0.00%
4971,ZICHRON MOSHE SCHOOL,Grade 4 Math,22,13.64%
4972,ZICHRON MOSHE SCHOOL,Grade 8 ELA,26,42.31%


In [65]:
# REPLACE THIS FILE ONCE YOU FIGURE OUT WHAT THE BASE FILE FOR VIOLATIONS SHOULD BE
data_violations = pd.read_csv('private_successful_matches.csv')
data_violations

,SchoolName,Address,ZipCode,LATITUDE,LONGITUDE,COUNTA of Level,Matched Score,Matched School,PSS_SCHOOL_ID,PSS_INST,...,PSS_HISP_PCT,PSS_BLACK_PCT,PSS_WHITE_PCT,PSS_PACISL_PCT,PSS_TWOMORE_PCT,PSS_STUDENT_TEACHER_RATIO,PSS_ORIENT,PSS_COUNTY_NAME,Matched Address,Matched School Name
0,ACADEMY OF MOUNT SAINT URSULA,330 BEDFORD PARK BLVD FL 1ST,10458.0,40.869088,-73.884837,0,95,ACADEMY OF MOUNT ST URSULA,00920855,ACADEMY OF MOUNT ST URSULA,...,63.66,26.98,2.76,0.00,1.38,15.62,1,BRONX,NaN,NaN
1,ACADEMY OF ST. DOROTHY,1305 HYLAN BLVD,10305.0,40.597122,-74.085236,3,98,ACADEMY OF ST DOROTHY,00920174,ACADEMY OF ST DOROTHY,...,3.38,3.95,85.31,0.00,0.00,18.63,1,RICHMOND,NaN,NaN
2,AHI EZER YESHIVA,2433 OCEAN PARKWAY,11235.0,40.590242,-73.965316,4,95,AHI EZER YESHIVA SCHOOL,00936162,AHI EZER YESHIVA SCHOOL,...,0.00,0.00,100.00,0.00,0.00,8.68,18,KINGS,NaN,NaN
3,AL IMAN SCHOOL,8989 VAN WYCK EXPY,11435.0,40.700908,-73.815220,7,100,AL-IMAN SCHOOL,A9104867,AL-IMAN SCHOOL,...,0.00,2.77,11.11,0.00,8.33,5.90,17,QUEENS,NaN,NaN
4,AL-IHSAN ACADEMY,130-08 ROCKAWAY BLVD,11420.0,40.674962,-73.808309,1,100,AL-IHSAN ACADEMY,A9300668,AL-IHSAN ACADEMY,...,0.00,4.96,17.25,0.00,0.00,11.13,17,QUEENS,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
652,THE SHIELD INSTITUTE,1800 ANDREWS AVE BSMT,10453.0,40.851283,-73.915951,0,86,NaN,BB101113,SHIELD INSTITUTE OF FLUSHING,...,47.82,23.91,13.04,0.00,2.17,6.57,30,QUEENS,NaN,SHIELD INSTITUTE OF FLUSHING
653,YESHIVA OF KASHO,101 SPENCER ST,11205.0,40.695661,-73.955384,0,86,NaN,A9702490,TALMUD TORAH OF KASHO,...,0.00,0.00,100.00,0.00,0.00,9.48,18,KINGS,NaN,TALMUD TORAH OF KASHO
654,YESHIVAH TIFERES TORAH,27 SKILLMAN ST,11205.0,40.697819,-73.957748,0,86,NaN,A1702684,TALMUD TORAH TIFERES YECHIEL D'ALEKSANDER INC,...,0.00,0.00,100.00,0.00,0.00,28.14,18,KINGS,NaN,TALMUD TORAH TIFERES YECHIEL D'ALEKSANDER INC
655,POLY PREP LOWER SCHOOL FS (PPL),50 PROSPECT PARK WEST,11215.0,40.669228,-73.972993,0,80,NaN,00937803,POLY PREP COUNTRY DAY SCHOOL,...,5.92,13.50,56.42,4.16,17.76,8.45,30,KINGS,NaN,POLY PREP COUNTRY DAY SCHOOL


In [66]:
from fuzzywuzzy import fuzz

fuzz.partial_ratio("OUR LADY OF FATIMA SCHOOL", "OUR LADY OF FATIMA")

100

In [67]:
data_tests[['School']].drop_duplicates()

,School
0,A FANTIS PAROCHIAL
10,ACADEMY OF ST DOROTHY
22,AHI EZER YESHIVA
34,AL MADRASA AL ISLAMIYA
46,AL-IHSAN ACADEMY
...,...
4918,YESHIVA-BROOKLYN
4941,YESHIVA-CENTRAL QUEENS
4953,YESHIVAT OHEL TORAH
4965,YESODEI BINA INC


In [84]:
def preprocess_schoolname(school_name):
    school_name = school_name.upper()
    school_name = school_name.replace('SCHOOL', '')
    school_name = school_name.strip()
    return school_name

def get_fuzzy_match(school_name):    

    # Fuzzy Match
    schools_with_test_scores = data_tests[['School']].drop_duplicates()
    schools_with_test_scores = schools_with_test_scores.dropna()
    schools_with_test_scores['fuzzymatch_score'] = schools_with_test_scores['School'].apply(lambda x: fuzz.ratio(
        preprocess_schoolname(x), 
        preprocess_schoolname(school_name)))
    schools_with_test_scores = schools_with_test_scores.sort_values(by='fuzzymatch_score', ascending=False)

    # get top school 
    top_school = schools_with_test_scores.iloc[0]['School']
    top_school_score = schools_with_test_scores.iloc[0]['fuzzymatch_score']

    # return
    return top_school, top_school_score


In [86]:
# assign fuzzymatch_name and fuzzymatch_score to data_violations['SchoolName'].apply(get_fuzzy_match)
data_violations['fuzzymatch_name'], data_violations['fuzzymatch_score'] = zip(*data_violations['SchoolName'].apply(get_fuzzy_match))
data_violations

,SchoolName,Address,ZipCode,LATITUDE,LONGITUDE,COUNTA of Level,Matched Score,Matched School,PSS_SCHOOL_ID,PSS_INST,...,PSS_WHITE_PCT,PSS_PACISL_PCT,PSS_TWOMORE_PCT,PSS_STUDENT_TEACHER_RATIO,PSS_ORIENT,PSS_COUNTY_NAME,Matched Address,Matched School Name,fuzzymatch_name,fuzzymatch_score
0,ACADEMY OF MOUNT SAINT URSULA,330 BEDFORD PARK BLVD FL 1ST,10458.0,40.869088,-73.884837,0,95,ACADEMY OF MOUNT ST URSULA,00920855,ACADEMY OF MOUNT ST URSULA,...,2.76,0.00,1.38,15.62,1,BRONX,NaN,NaN,ACADEMY OF ST DOROTHY,56
1,ACADEMY OF ST. DOROTHY,1305 HYLAN BLVD,10305.0,40.597122,-74.085236,3,98,ACADEMY OF ST DOROTHY,00920174,ACADEMY OF ST DOROTHY,...,85.31,0.00,0.00,18.63,1,RICHMOND,NaN,NaN,ACADEMY OF ST DOROTHY,98
2,AHI EZER YESHIVA,2433 OCEAN PARKWAY,11235.0,40.590242,-73.965316,4,95,AHI EZER YESHIVA SCHOOL,00936162,AHI EZER YESHIVA SCHOOL,...,100.00,0.00,0.00,8.68,18,KINGS,NaN,NaN,AHI EZER YESHIVA,100
3,AL IMAN SCHOOL,8989 VAN WYCK EXPY,11435.0,40.700908,-73.815220,7,100,AL-IMAN SCHOOL,A9104867,AL-IMAN SCHOOL,...,11.11,0.00,8.33,5.90,17,QUEENS,NaN,NaN,AL-IMAN SCHOOL,86
4,AL-IHSAN ACADEMY,130-08 ROCKAWAY BLVD,11420.0,40.674962,-73.808309,1,100,AL-IHSAN ACADEMY,A9300668,AL-IHSAN ACADEMY,...,17.25,0.00,0.00,11.13,17,QUEENS,NaN,NaN,AL-IHSAN ACADEMY,100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
652,THE SHIELD INSTITUTE,1800 ANDREWS AVE BSMT,10453.0,40.851283,-73.915951,0,86,NaN,BB101113,SHIELD INSTITUTE OF FLUSHING,...,13.04,0.00,2.17,6.57,30,QUEENS,NaN,SHIELD INSTITUTE OF FLUSHING,JEWISH INSTITUTE-QUEENS,60
653,YESHIVA OF KASHO,101 SPENCER ST,11205.0,40.695661,-73.955384,0,86,NaN,A9702490,TALMUD TORAH OF KASHO,...,100.00,0.00,0.00,9.48,18,KINGS,NaN,TALMUD TORAH OF KASHO,YESHIVA OHEL MOSHE,71
654,YESHIVAH TIFERES TORAH,27 SKILLMAN ST,11205.0,40.697819,-73.957748,0,86,NaN,A1702684,TALMUD TORAH TIFERES YECHIEL D'ALEKSANDER INC,...,100.00,0.00,0.00,28.14,18,KINGS,NaN,TALMUD TORAH TIFERES YECHIEL D'ALEKSANDER INC,YESHIVA DARCHEI TORAH,74
655,POLY PREP LOWER SCHOOL FS (PPL),50 PROSPECT PARK WEST,11215.0,40.669228,-73.972993,0,80,NaN,00937803,POLY PREP COUNTRY DAY SCHOOL,...,56.42,4.16,17.76,8.45,30,KINGS,NaN,POLY PREP COUNTRY DAY SCHOOL,ST PETER & PAUL,45


In [88]:
test_df = data_violations[['SchoolName', 'Address', 'fuzzymatch_name', 'fuzzymatch_score']].sort_values(by='fuzzymatch_score', ascending=False)
test_df
# test_df.to_csv('test.csv', index=False)

,SchoolName,Address,fuzzymatch_name,fuzzymatch_score
381,PHYL'S ACADEMY PREP SCHOOL,3520 TILDEN AVE,PHYL'S ACADEMY PREP,100
575,TALMUD TORAH OHR MOSHE,1676 61 ST,TALMUD TORAH OHR MOSHE,100
592,INCARNATION SCHOOL,570 W 175 ST,INCARNATION SCHOOL,100
370,OUR LADY OF FATIMA SCHOOL,2538 80 ST,OUR LADY OF FATIMA,100
467,YESHIVA OHEL MOSHE,7914 BAY PARKWAY,YESHIVA OHEL MOSHE,100
...,...,...,...,...
54,HEWITT SCHOOL,45 E 75 ST,TREY WHITFIELD SCHOOL,40
606,HEWITT SCHOOL,45 E 75 ST,TREY WHITFIELD SCHOOL,40
102,NIGHTINGALE-BAMFORD SCHOOL FS (NBS),20 E 92 ST,NOTRE DAME JR-SR HS,38
642,NIGHTINGALE-BAMFORD SCHOOL FS (NBS),20 E 92 ST,NOTRE DAME JR-SR HS,38
